In [1]:
from functools import partial
from sqlite3 import connect

import astroplan as ap
import astropy.units as u
from icecream import ic, install

import astropaul.targetlistcreator as tlc
import astropaul.html as html
import astropaul.phase as ph
import astropaul.priority as pr

install() # icecream will now work in imported packages

%load_ext autoreload
%autoreload 2


In [2]:
conn = connect("../../django/TargetDB/db.sqlite3")

session = tlc.ObservingSession(ap.Observer.at_site("APO"))
session.add_full_day("2025-06-07")
session.add_full_day("2025-06-08")
session.add_full_day("2025-06-10")
session.add_full_day("2025-06-11")
session.add_half_day("2025-06-12", first_half=False)

phase_event_defs = [
    ph.PhaseEventDef("Not in Eclipse", partial(ph.calc_time_of_gress, ingress=False)),
    ph.PhaseEventDef("Eclipse", partial(ph.calc_time_of_gress, ingress=True)),
]

min_altitude = 35 * u.deg
max_magnitude = 13

name = "APO Observing List 2025-06-07 to -12"
creator = tlc.TargetListCreator(name=name, connection=conn)
creator.steps = [
    tlc.add_targets,
    partial(tlc.filter_targets, criteria=lambda df: (df["Target Type"].isin(["QuadEB", "SextEB"]))),
    tlc.add_tess,
    tlc.add_coords,
    partial(tlc.hide_cols, prefix="TESS "),
    partial(
        tlc.add_observability,
        observing_session=session,
        calc_moon_distance=True,
        observability_threshold=(min_altitude, 80 * u.deg),
    ),
    partial(tlc.filter_targets, criteria=lambda df: (df["Observable Any Night"])),
    partial(tlc.filter_targets, criteria=lambda df: (df["Vmag"] < max_magnitude)),
    tlc.add_ephemerides,
    partial(tlc.add_phase_events, observing_session=session, phase_event_defs=phase_event_defs),
    tlc.add_speckle,
    # partial(tlc.add_speckle_phase, phase_event_defs=phase_event_defs),
    partial(tlc.add_side_status, phase_event_defs=phase_event_defs),
]
tl = creator.calculate(verbose=False)

# html.render_observing_pages(tl, None, {}, "html")
print(tl.summarize())

APO Observing List 2025-06-07 to -12
Criteria
    lambda df: (df["Target Type"].isin(["QuadEB", "SextEB"]))),
    Observability threshold: (<Quantity 35. deg>, <Quantity 80. deg>)
    AltitudeConstraint: {'min': np.float64(35.0), 'max': np.float64(80.0), 'boolean_constraint': True}
    lambda df: (df["Observable Any Night"])),
    lambda df: (df["Vmag"] < 13)),
35 targets:
      35 QuadEB
Column Count (primary, secondary):
    Target: (1, 2)
    Coordinates: (6, 3)
    Observable: (5, 20)
    Count: (1, 0)
Associated tables:
       5 rows,  2 columns: Lunar Phases
     117 rows,  7 columns: Ephemerides
     752 rows, 10 columns: Phase Events
      76 rows,  6 columns: Speckle Observations
      29 rows,  6 columns: SIDE Observations



In [4]:
altitude_categories = [
    ((-90, min_altitude.value), 0),
    ((min_altitude.value, 90), 1),
]

pl = pr.PriorityList(tl, session, interval=30 * u.min)
pr.calculate_altitude_priority(pl, altitude_categories=altitude_categories)
pr.prioritize_phase_sequence(pl, ["Eclipse"], "Eclipse", True, True, True)
pr.prioritize_side_observation(pl, side_state="Eclipse")
pr.calculate_overall_priority(pl)
pr.aggregate_target_priorities(pl, skip_column_threshold=0.1)
pl.categorize_priorities(bins=[0.00, 0.20, 0.40, 0.6, 1.00], labels=["", "*", "* *", "* * *"])

html.render_observing_pages(tl, pl, {}, "../../Observing Files/APO Observing 2025-06-07 to -12")
# pl.categorical_priorities[0]